# LE Hyperparameter Search (`lr` / epochs)

This notebook runs a Localized Entropy (LE) hyperparameter search using `configs/default.json` as-is.

- Outer loop: learning rate (`lr`) starts at `1.0` and halves each step.
- Inner loop: train for `10` epochs and evaluate each epoch.
- Per-epoch collection:
  - LE loss on test/eval split
  - Global calibration ratio on eval/test split only
  - Per-condition calibration ratio on eval/test split only


In [ ]:
%matplotlib inline
from __future__ import annotations

import math

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from localized_entropy.hyper_search import build_search_context, run_le_hyper_search

np.set_printoptions(precision=6, suppress=True)
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 50)

CONFIG_PATH = "configs/hyper.json"
ctx = build_search_context(CONFIG_PATH)

print(f"Evaluation split for LE loss: {ctx.eval_name}")
print(f"Data source: {ctx.data_source}")
print(f"Num conditions/categories: {ctx.splits.num_conditions}")



In [ ]:
def dropout_schedule_doubling(start: float = 0.01, max_value: float = 0.8):
    values = []
    dropout = float(start)
    max_value = float(max_value)
    while dropout <= max_value:
        values.append(dropout)
        dropout = dropout * 2.0
    return values


DROPOUT_VALUES = dropout_schedule_doubling(start=0.01, max_value=0.8)
param_grid = [{"dropout": float(v)} for v in DROPOUT_VALUES]

print(f"Number of dropout points: {len(DROPOUT_VALUES)}")
print("dropout first/last:", DROPOUT_VALUES[0], DROPOUT_VALUES[-1])



In [ ]:
def apply_params(cfg_run: dict, train_params: dict, params: dict) -> None:
    cfg_run.setdefault("model", {})
    cfg_run["model"]["dropout"] = float(params["dropout"])


results_df, condition_df = run_le_hyper_search(
    ctx,
    param_grid=param_grid,
    apply_params=apply_params,
    record_params=lambda p: {"dropout": float(p["dropout"])},
    sort_by=["test_le", "dropout", "epoch"],
    ascending=[True, True, True],
)

best_row = results_df.iloc[0]
print("Best LE result overall:")
print(best_row.to_dict())



In [ ]:
def plot_metric_lines(metric_df: pd.DataFrame, value_col: str, title: str, y_label: str) -> None:
    plt.figure(figsize=(11, 6))
    dropout_order = sorted(metric_df["dropout"].unique(), reverse=True)
    for dropout in dropout_order:
        block = metric_df.loc[metric_df["dropout"] == dropout, ["epoch", value_col]].sort_values("epoch")
        plt.plot(
            block["epoch"],
            block[value_col],
            marker="o",
            linewidth=1.5,
            markersize=3,
            label=f"dropout={dropout:.6g}",
        )
    plt.xlabel("Epoch")
    plt.ylabel(y_label)
    plt.title(title)
    plt.grid(True, alpha=0.3)
    plt.legend(ncol=3, fontsize=8)
    if "calibration" in value_col.lower():
        plt.ylim(0, 3)
    plt.tight_layout()
    plt.show()


table_cols = ["dropout", "epoch", "test_le", "global_calibration"]
results_sorted = results_df.sort_values(["test_le", "dropout", "epoch"], ascending=[True, True, True]).reset_index(drop=True)
condition_sorted = condition_df.sort_values(["condition", "dropout", "epoch"]).reset_index(drop=True)

display(results_sorted.loc[:, table_cols].head(100))

best_row = results_sorted.iloc[0]
print("Most optimal dropout (by minimum test_le):", float(best_row["dropout"]))
print("Best epoch for that dropout:", int(best_row["epoch"]))
print("Best test_le:", float(best_row["test_le"]))

plot_metric_lines(
    metric_df=results_sorted,
    value_col="test_le",
    title=f"LE ({ctx.eval_name}) across dropout and epoch",
    y_label="LE loss",
)

plot_metric_lines(
    metric_df=results_sorted,
    value_col="global_calibration",
    title="Global calibration ratio across dropout and epoch",
    y_label="Calibration ratio",
)

condition_table_cols = ["dropout", "epoch", "condition", "count", "base_rate", "pred_mean", "calibration"]
display(condition_sorted.loc[:, condition_table_cols])

if condition_sorted.empty:
    print("No per-condition records to plot.")
else:
    cond_ids = sorted(condition_sorted["condition"].unique())
    n = len(cond_ids)
    ncols = 3
    nrows = int(math.ceil(n / ncols))
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5 * ncols, 3.8 * nrows))
    axes = np.array(axes).reshape(-1)

    dropout_order = sorted(condition_sorted["dropout"].unique(), reverse=True)
    epoch_order = sorted(condition_sorted["epoch"].unique())

    for i, cond_id in enumerate(cond_ids):
        ax = axes[i]
        block = condition_sorted.loc[condition_sorted["condition"] == cond_id, ["dropout", "epoch", "calibration"]]
        ax.set_title(f"Condition {cond_id}")
        for dropout in dropout_order:
            dropout_block = block.loc[block["dropout"] == dropout, ["epoch", "calibration"]].sort_values("epoch")
            if dropout_block.empty:
                continue
            ax.plot(
                dropout_block["epoch"],
                dropout_block["calibration"],
                marker="o",
                linewidth=1.2,
                markersize=2.5,
                label=f"{dropout:.3g}",
            )
        ax.set_xticks(epoch_order)
        ax.set_xlabel("Epoch")
        ax.set_ylabel("Calibration")
        ax.set_ylim(0, 3)
        ax.grid(True, alpha=0.3)
        ax.legend(fontsize=6)

    for j in range(len(cond_ids), len(axes)):
        axes[j].axis("off")

    fig.suptitle("Per-condition calibration ratio across epoch (line per dropout)")
    fig.tight_layout()
    plt.show()

